In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [15]:
df=pd.read_csv('/content/sample_data/support_ticket_routing.csv')

In [16]:
df

,TicketID,Department,Description
0,101,IT Support,Laptop not starting VPN not connecting email i...
1,102,HR,Leave balance salary slip update and attendanc...
2,103,Finance,Invoice mismatch payment pending and tax deduc...
3,104,Sales,Client asking about product pricing discount a...
4,105,IT Support,System slow password reset software installati...
5,106,HR,Payslip missing salary not credited PF deducti...


In [23]:
#new_email='My salary is not creadited yet and payslip is missing'
user_email=input('Enter the email')
new_email=user_email

Enter the emailMy laptop is not working , I think RAM issue


In [24]:
docuemnts=df['Description'].tolist()+[new_email]

In [25]:
vectorizer=TfidfVectorizer(stop_words='english')

In [26]:
tfidf_matrix=vectorizer.fit_transform(docuemnts)

In [27]:
similarity_matrix=cosine_similarity(tfidf_matrix[:-1],
                                    tfidf_matrix[-1])
df['SimilarityScore']=similarity_matrix.flatten()

print(df[['TicketID',	'Department','SimilarityScore']])



   TicketID  Department  SimilarityScore
0       101  IT Support         0.235301
1       102          HR         0.076371
2       103     Finance         0.076371
3       104       Sales         0.000000
4       105  IT Support         0.000000
5       106          HR         0.000000


In [28]:
best_ticket=df.loc[df['SimilarityScore'].idxmax()]
print('Send this email to department')
print(best_ticket['Department'])

Send this email to department
IT Support


## Day-21 Exericse-01
* An online learning platform reommends the best course based on learners requirement.
* DataSet and learner requirement as follows.
* dataset={
    'Course Id':[1,2,3,4,5],
    'Course Name':['Python Basic','AI-Machine Learning','Web Development','Data Anlytics','Data Science'],
    'Course Description':[
        'Python for beginners covering basics of python loops,oops,functions datatype etc.',
        'Advanced machine learning, deep learning ai agent with tensorflow',
        'Web development using html, css,javascript, angular ',
        'Data Analysis with python, pandas, numpy and matplotlip visualization',
        'Data Cleaning, Power BI, MS Excel, MS Access and SQL Server , Normalization , Denormalization'
    ]
}
* learner_query='I want to learn python for Data Anlaysis and visualization'
* Apply TF-IDF on DataFrame Text and find out based course for learner.


In [30]:
dataset={
    'Course Id':[1,2,3,4,5],
    'Course Name':['Python Basic','AI-Machine Learning','Web Development','Data Anlytics','Data Science'],
    'Course Description':[
        'Python for beginners covering basics of python loops,oops,functions datatype etc.',
        'Advanced machine learning, deep learning ai agent with tensorflow',
        'Web development using html, css,javascript, angular ',
        'Data Analysis with python, pandas, numpy and matplotlip visualization',
        'Data Cleaning, Power BI, MS Excel, MS Access and SQL Server , Normalization , Denormalization'
    ]
}

In [32]:
df_courses=pd.DataFrame(dataset)
df_courses

,Course Id,Course Name,Course Description
0,1,Python Basic,Python for beginners covering basics of python...
1,2,AI-Machine Learning,"Advanced machine learning, deep learning ai ag..."
2,3,Web Development,"Web development using html, css,javascript, an..."
3,4,Data Anlytics,"Data Analysis with python, pandas, numpy and m..."
4,5,Data Science,"Data Cleaning, Power BI, MS Excel, MS Access a..."


In [ ]:
#NLP : Model Logisitic Regression


In [33]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [40]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [37]:
dataset={
    'EmailText':['Win a free IPhone click the link below',
                 'Limited time offer claim your prize of 20000 USD',
                 'Meeting Schedule for tommorow',
                 'Please find attached business report',
                 'Congratulation!!! your have won a lottery of RM 40000',
                 'Let us meet and discussed project',
                 'Your name came in lucky draw send account details for getting free money'
                 ],
    'IsSpam':[1,1,0,0,1,0,1]  #Spam =1 , Not Spam=0
}



In [38]:
df=pd.DataFrame(dataset)
print(df)

                                           EmailText  IsSpam
0             Win a free IPhone click the link below       1
1   Limited time offer claim your prize of 20000 USD       1
2                      Meeting Schedule for tommorow       0
3               Please find attached business report       0
4  Congratulation!!! your have won a lottery of R...       1
5                  Let us meet and discussed project       0
6  Your name came in lucky draw send account deta...       1


In [41]:
stop_words=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()

In [42]:
#Text Cleaning and Lemmatization
def clean_text(text):
  text=text.lower()
  text=re.sub(r'[^a-zA-Z0-9\s]','',text)
  words=text.split()
  words=[lemmatizer.lemmatize(word) for word in words if word not in stop_words]
  cleaned_text=' '.join(words)
  return cleaned_text


In [44]:
df['CleanedEmail']=df['EmailText'].apply(clean_text)
df

,EmailText,IsSpam,CleanedEmail
0,Win a free IPhone click the link below,1,win free iphone click link
1,Limited time offer claim your prize of 20000 USD,1,limited time offer claim prize 20000 usd
2,Meeting Schedule for tommorow,0,meeting schedule tommorow
3,Please find attached business report,0,please find attached business report
4,Congratulation!!! your have won a lottery of R...,1,congratulation lottery rm 40000
5,Let us meet and discussed project,0,let u meet discussed project
6,Your name came in lucky draw send account deta...,1,name came lucky draw send account detail getti...


In [45]:
vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(df['CleanedEmail'])
y=df['IsSpam']

In [47]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [48]:
model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [49]:
y_pred=model.predict(X_test)
#
print('Actual values',y_test.values)
print('Predicted values',y_pred)

Actual values [1]
Predicted values [1]


In [50]:
print('Accuracy: ',accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))
#

Accuracy:  1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [53]:
#Predict a new email is spam or not a spam using this model
new_email=input('Enter the Email')
cleaned_new_email=clean_text(new_email)
vectorized_new_email=vectorizer.transform([cleaned_new_email])
prediction=model.predict(vectorized_new_email)
if prediction[0]==1:
  print('Spam')
else:
  print('Not Spam')

Enter the EmailMeeting at 3
Not Spam
